This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

[back to overview](kochanek-bartels.ipynb)

# Non-Uniform Kochanek--Bartels Splines

<cite data-cite="kochanek1984tcb">Kochanek and Bartels (1984)</cite>
mainly talks about uniform splines.
Only in section 4 -- "Adjustments for Parameter Step Size" --
the authors briefly mention the non-uniform case
and provide equations for "adjusted tangent vectors":

> The formulas
> [...]
> assume an equal time spacing of key frames,
implying an equal number of inbetweens within each key
interval. A problem can exist if the animator requests a
different number of inbetweens for adjacent intervals.
> [...]
> If the
same parametric derivative is used for both splines at $P_i$, these
different step sizes will cause a discontinuity in the speed of
motion. What is required, if this discontinuity is not intentional,
is a means of making a local adjustment to the interval
separating successive frames before and after the key frame so
that the speed of entry matches the speed of exit. This can be
accomplished by adjusting the specification of the tangent
vector at the key frame based on the number of inbetweens in
the adjacent intervals.
> [...]
> Once the
tangent vectors have been found for an equal number of
inbetweens in the adjacent intervals, the adjustment required for
different numbers of inbetweens ($N_{i-1}$ frames between $P_{i-1}$ and
$P_i$ followed by $N_i$ frames between $P_i$ and $P_{i+1}$) can be made by
weighting the tangent vectors appropriately:
>
> \begin{align*}
\text{adjusted } DD_i &=
DD_i
\frac{2 N_{i-1}}{N_{i-1} + N_i}\\
\text{adjusted } DS_i &=
DS_i
\frac{2 N_i}{N_{i-1} + N_i}
\end{align*}
>
> ---<cite data-cite="kochanek1984tcb">Kochanek and Bartels (1984)</cite>, section 4, "Adjustments for Parameter Step Size"

In their notation,
$DS_i$ is the *source derivative*
(i.e. the *incoming* tangent vector)
at point $P_i$,
and
$DD_i$ is the *destination derivative*
(i.e. the *outgoing* tangent vector).
The point $P_i$ corresponds to $\boldsymbol{x}_i$ in our notation.

To be able to play around with that,
let's implement it in a function.
It turns out that for the way we will be using this function,
we have to use the reciprocal value of the adjustment mentioned in the paper:

In [ ]:
def kochanek_bartels_tangents(xs, ns):
    """Adjusted tangent vectors according to Kochanek & Bartels."""
    x_1, _, x1 = xs
    N_1, N0 = ns
    uniform = (x1 - x_1) / 2
    # NB: the K&B paper uses reciprocal weighting factors:
    incoming = uniform * (N_1 + N0) / (2 * N0)
    outgoing = uniform * (N_1 + N0) / (2 * N_1)
    return incoming, outgoing

We can see that the uniform tangents are re-scaled
but their direction is unchanged.

This is a hint that
-- although the paper claims to be using Catmull--Rom splines --
we'll get different results than in the
[notebooks about Catmull--Rom splines](catmull-rom-properties.ipynb#Tangent-Vectors).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

We'll need the Hermite basis matrix
that we derived in the
[notebook about uniform Hermite splines](hermite-uniform.ipynb#Basis-Matrix)
and which is also shown in equation 2 of
<cite data-cite="kochanek1984tcb">Kochanek and Bartels (1984)</cite>:

In [ ]:
hermite_matrix = np.array([
    [ 2, -2,  1,  1],
    [-3,  3, -2, -1],
    [ 0,  0,  1,  0],
    [ 1,  0,  0,  0]])

Since the paper uses a different (implicit) re-scaling of parameter values
(based on the numbers of *inbetweens*),
we cannot use the classes from the
[splines](../python-module/splines.rst) module
and have to re-implement everything from scratch:

In [ ]:
def pseudo_catmull_rom(xs, ns):
    """Closed Catmull-Rom spline according to Kochanek & Bartels."""
    xs = np.asarray(xs)
    L = len(xs)
    assert L >= 2
    assert L == len(ns)
    tangents = [
        tangent
        for i in range(L)
        for tangent in kochanek_bartels_tangents(
            [xs[i], xs[(i + 1) % L], xs[(i + 2) % L]],
            [ns[i], ns[(i + 1) % L]])
    ]
    # Move last (outgoing) tangent to the beginning:
    tangents = tangents[-1:] + tangents[:-1]
    ts = [
        np.linspace(0, 1, n + 1, endpoint=False).reshape(-1, 1)
        for n in ns]
    return np.concatenate([
        t**[3, 2, 1, 0] @ hermite_matrix @ [xs[i], xs[(i + 1) % L], v0, v1]
        for i, (t, v0, v1)
        in enumerate(zip(ts, tangents[::2], tangents[1::2]))])

<div class="alert alert-info">

Note

The `@` operator is used here to do
[NumPy's matrix multiplication](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html).

</div>

Let's plot an example:

In [ ]:
vertices1 = [
    (0, 0),
    (1, 1),
    (2, 0),
]
inbetweens1 = [
    5,
    20,
    15,
]

In [ ]:
plt.scatter(*pseudo_catmull_rom(vertices1, inbetweens1).T, marker='.')
plt.scatter(*np.array(vertices1).T, marker='x', color='k')
plt.axis('equal');

This doesn't look too bad,
let's plot the same thing with the
[splines.CatmullRom](../python-module/splines.rst#splines.CatmullRom)
class for comparison.

In [ ]:
from splines import CatmullRom

In oder to be able to compare the results,
we have to convert the discrete numbers of *inbetweens*
into re-scaled parameter values:

In [ ]:
def inbetweens2times(inbetweens):
    inbetweens = np.asarray(inbetweens)
    inbetweens += 1
    return np.cumsum([0, *inbetweens])

In [ ]:
times1 = inbetweens2times(inbetweens1)

Now we have everything to create a
non-uniform Catmull--Rom spline ...

In [ ]:
cr_spline1 = CatmullRom(vertices1, times1, endconditions='closed')

... and with a helper function from [helper.py](helper.py) ...

In [ ]:
from helper import plot_spline_2d

... we can plot it for direct comparison
with the one suggested by Kochanek and Bartels:

In [ ]:
plt.plot(
    *pseudo_catmull_rom(vertices1, inbetweens1).T,
    marker='.', linestyle='', label='K&B')
plot_spline_2d(cr_spline1, dots_per_second=1, label='ours')
plt.legend(numpoints=3);

Here we can clearly see that not only the lengths of the tangent vectors
but also their directions have been adjusted
according to the neighboring parameter intervals.

Let's look at a different example:

In [ ]:
vertices2 = [
    (0, 0),
    (0, 0.5),
    (4.5, 1.5),
    (5, 1),
    (2, -1),
    (1.5, -1),
]
inbetweens2 = [
    2,
    15,
    3,
    12,
    2,
    10,
]

In [ ]:
times2 = inbetweens2times(inbetweens2)

In [ ]:
cr_spline2 = CatmullRom(vertices2, times2, endconditions='closed')

In [ ]:
plt.plot(
    *pseudo_catmull_rom(vertices2, inbetweens2).T,
    marker='.', linestyle='', label='K&B')
plot_spline_2d(cr_spline2, dots_per_second=1, label='ours')
plt.legend(numpoints=3);

This should illustrate the shortcomings of the tangent vectors
suggested by Kochanek and Bartels.

Instead of sticking with their suggestion,
we base the rest of our calculations
on the correct expression for
[tangent vectors of non-uniform Catmull--Rom splines](catmull-rom-non-uniform.ipynb#Tangent-Vectors):

\begin{equation*}
\boldsymbol{\dot{x}}_i =
\frac{
(t_{i+1} - t_i)^2 (\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
(t_i - t_{i-1})^2 (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{
(t_{i+1} - t_i)(t_i - t_{i-1})(t_{i+1} - t_{i-1})
}
\end{equation*}

## Parameters

In general,
the incoming tangent $\boldsymbol{\dot{x}}_i^{(-)}$ and
the outgoing tangent $\boldsymbol{\dot{x}}_i^{(+)}$
at vertex $\boldsymbol{x}_i$
can be written as:

\begin{align*}
a_i &= (1 - T_i) (1 + C_i) (1 + B_i)\\
b_i &= (1 - T_i) (1 - C_i) (1 - B_i)\\
c_i &= (1 - T_i) (1 - C_i) (1 + B_i)\\
d_i &= (1 - T_i) (1 + C_i) (1 - B_i)
\end{align*}

\begin{align*}
\boldsymbol{\dot{x}}_i^{(+)} &= \frac{
a_i (t_{i+1} - t_i)^2 (\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
b_i (t_i - t_{i-1})^2 (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{(t_{i+1} - t_i) (t_i - t_{i-1}) (t_{i+1} - t_{i-1})}\\
\boldsymbol{\dot{x}}_i^{(-)} &= \frac{
c_i (t_{i+1} - t_i)^2 (\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
d_i (t_i - t_{i-1})^2 (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{(t_{i+1} - t_i) (t_i - t_{i-1}) (t_{i+1} - t_{i-1})}
\end{align*}

In the calculation below, we consider -- without loss of generality --
the outgoing tangent at $\boldsymbol{x}_4$ and
the incoming tangent at $\boldsymbol{x}_5$.

\begin{align*}
a_4 &= (1 - T_4) (1 + C_4) (1 + B_4)\\
b_4 &= (1 - T_4) (1 - C_4) (1 - B_4)\\
c_5 &= (1 - T_5) (1 - C_5) (1 + B_5)\\
d_5 &= (1 - T_5) (1 + C_5) (1 - B_5)
\end{align*}

\begin{align*}
\boldsymbol{\dot{x}}_4^{(+)} &= \frac{
a_4 (t_5 - t_4)^2 (\boldsymbol{x}_4 - \boldsymbol{x}_3) +
b_4 (t_4 - t_3)^2 (\boldsymbol{x}_5 - \boldsymbol{x}_4)
}{(t_5 - t_4) (t_4 - t_3) (t_5 - t_3)}\\
\boldsymbol{\dot{x}}_5^{(-)} &= \frac{
c_5 (t_6 - t_5)^2 (\boldsymbol{x}_5 - \boldsymbol{x}_4) +
d_5 (t_5 - t_4)^2 (\boldsymbol{x}_6 - \boldsymbol{x}_5)
}{(t_6 - t_5) (t_5 - t_4) (t_6 - t_4)}
\end{align*}

## Calculation

As we have done before,
we are using [SymPy](https://www.sympy.org/) ...

In [ ]:
import sympy as sp
sp.init_printing()

... with a few helper classes from [utility.py](utility.py):

In [ ]:
from utility import NamedExpression, NamedMatrix

We are looking at the fifth spline segment
from $\boldsymbol{x}_4$ to $\boldsymbol{x}_5$
for $t_4 \le t \le t_5$.
However, we also need the preceding and following vertex
$\boldsymbol{x}_3$ and $\boldsymbol{x}_6$,
together with their respective parameter values $t_3$ and $t_6$:

In [ ]:
x3, x4, x5, x6 = sp.symbols('xbm3:7')

In [ ]:
t, t3, t4, t5, t6 = sp.symbols('t t3:7')

We are using the same control values as in the
[uniform case](kochanek-bartels-uniform.ipynb#Calculation) ...

In [ ]:
control_values_KB = sp.Matrix([x3, x4, x5, x6])
control_values_KB

... and the same additional TCB parameters per vertex.
Again,
only the parameters belonging to $\boldsymbol{x}_4$ and $\boldsymbol{x}_5$
are relevant:

In [ ]:
T4, T5 = sp.symbols('T4 T5')
C4, C5 = sp.symbols('C4 C5')
B4, B5 = sp.symbols('B4 B5')

In [ ]:
a4 = NamedExpression('a4', (1 - T4) * (1 + C4) * (1 + B4))
b4 = NamedExpression('b4', (1 - T4) * (1 - C4) * (1 - B4))
c5 = NamedExpression('c5', (1 - T5) * (1 - C5) * (1 + B5))
d5 = NamedExpression('d5', (1 - T5) * (1 + C5) * (1 - B5))
display(a4, b4, c5, d5)

In [ ]:
xd4 = NamedExpression(
    'xdotbm4^(+)',
    (a4.name * (t5 - t4)**2 * (x4 - x3) + b4.name * (t4 - t3)**2 * (x5 - x4)) /
    ((t5 - t4) * (t4 - t3) * (t5 - t3)))
xd5 = NamedExpression(
    'xdotbm5^(-)',
    (c5.name * (t6 - t5)**2 * (x5 - x4) + d5.name * (t5 - t4)**2 * (x6 - x5)) /
    ((t6 - t5) * (t5 - t4) * (t6 - t4)))
display(xd4, xd5)

In [ ]:
display(xd4.subs_symbols(a4, b4))
display(xd5.subs_symbols(c5, d5))

Like in the uniform case,
we try to find a transformation from the given control values
to Hermite control values:

In [ ]:
control_values_H = sp.Matrix([x4, x5, xd4.name, xd5.name])
M_KBtoH = NamedMatrix(r'{M_{\text{KB$,4\to$H},4}}', 4, 4)
NamedMatrix(control_values_H, M_KBtoH.name * control_values_KB)

With the basis matrix from the
[notebook about non-uniform Hermite splines](hermite-non-uniform.ipynb#Basis-Matrix) ...

In [ ]:
M_H = NamedMatrix(
    r'{M_{\text{H},4}}',
    sp.Matrix([[ 2, -2,  1,  1],
               [-3,  3, -2, -1],
               [ 0,  0,  1,  0],
               [ 1,  0,  0,  0]]) * sp.diag(1, 1, t5 - t4, t5 - t4))
M_H

... this can then be used to get the full basis matrix:

In [ ]:
M_KB = NamedMatrix(r'{M_{\text{KB},4}}', M_H.name * M_KBtoH.name)
M_KB

If we substitute the above definitions of
$\boldsymbol{\dot{x}}_4^{(+)}$ and
$\boldsymbol{\dot{x}}_5^{(-)}$,
we can obtain the matrix elements:

In [ ]:
M_KBtoH.expr = sp.Matrix([
    [expr.coeff(cv).simplify() for cv in control_values_KB]
    for expr in control_values_H.subs([xd4.args, xd5.args]).expand()])
M_KBtoH

In [ ]:
delta3, delta4, delta5 = sp.symbols('Delta3:6')
deltas = {
    t3: 0,
    t4: delta3,
    t5: delta3 + delta4,
    t6: delta3 + delta4 + delta5,
}

In [ ]:
M_KBtoH.subs(deltas).simplify()

In [ ]:
M_KB = M_KB.subs_symbols(M_H, M_KBtoH).doit()
M_KB.subs(deltas).expand()

And for completeness' sake, its inverse:

In [ ]:
M_KB.subs(deltas).expand().I